In [ ]:
# -*- coding: utf-8 -*-
# 匯出 Champion 模型 (LGBM, 用 onnxmltools) 與 Baseline (LogReg, 用 skl2onnx) 為 ONNX

# 安裝必要套件
!pip install -q onnxmltools skl2onnx onnxruntime lightgbm

from pathlib import Path
import joblib
import numpy as np
import onnxruntime as ort
import pandas as pd
from sklearn.linear_model import LogisticRegression

# 匯出工具
from onnxmltools.convert import convert_lightgbm
from onnxmltools.convert.common.data_types import FloatTensorType as FloatTensorType_onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType as FloatTensorType_skl

# ========== 0) 路徑 ==========
ART_LGBM = Path("./artifacts_lgbm")
ART_ONNX = Path("./onnx_models")
ART_ONNX.mkdir(parents=True, exist_ok=True)

# ========== 1) 載入模型 ==========
# LGBM Champion
lgbm_model = joblib.load(ART_LGBM / "lgbm_model.pkl")
scaler = joblib.load(ART_LGBM / "scaler.pkl")

# Logistic Regression Baseline (重新訓練)
feat = pd.read_parquet("./data/feat_6h.parquet")
label = pd.read_parquet("./data/label_6h.parquet")
df = feat.merge(label, on="ts_utc", how="inner").sort_values("ts_utc")

X = df.drop(columns=["ts_utc","y_dir_6h","y_tail_6h"]).select_dtypes(include=[np.number]).values
y = df["y_dir_6h"].astype(int).values
X_scaled = scaler.transform(X)

logreg_model = LogisticRegression(max_iter=200, solver="lbfgs")
logreg_model.fit(X_scaled, y)

# ========== 2) 匯出 LGBM ==========
feature_dim = scaler.mean_.shape[0]
onnx_lgbm = convert_lightgbm(
    lgbm_model.booster_,
    initial_types=[('input', FloatTensorType_onnx([None, feature_dim]))],
    target_opset=15
)
with open(ART_ONNX / "lgbm_model.onnx", "wb") as f:
    f.write(onnx_lgbm.SerializeToString())
print("[OK] LGBM 匯出完成 -> lgbm_model.onnx")

# ========== 3) 匯出 Logistic Regression ==========
onnx_logreg = convert_sklearn(
    logreg_model,
    initial_types=[('input', FloatTensorType_skl([None, feature_dim]))]
)
with open(ART_ONNX / "logreg_model.onnx", "wb") as f:
    f.write(onnx_logreg.SerializeToString())
print("[OK] Logistic Regression 匯出完成 -> logreg_model.onnx")

# ========== 4) 驗證 ONNX 模型 ==========
X_dummy = scaler.transform(np.random.rand(1, feature_dim))

for name, model_path in [("LGBM", "lgbm_model.onnx"), ("LogReg", "logreg_model.onnx")]:
    sess = ort.InferenceSession(str(ART_ONNX / model_path))
    input_name = sess.get_inputs()[0].name
    pred = sess.run(None, {input_name: X_dummy.astype(np.float32)})[0]
    print(f"[Check] {name} 輸出機率: {pred}")

